In [13]:
import ast
import pickle
import pandas as pd
# Load the TF-IDF matrix from the file
with open("fe_data/tfidf_X_title.pkl", "rb") as f:
    x_title = pickle.load(f)

with open("fe_data/tfidf_X_content.pkl", "rb") as f:
    x_content = pickle.load(f)
    

print(x_title)


  (0, 1433)	0.33482711194677267
  (0, 970)	0.33482711194677267
  (0, 509)	0.33482711194677267
  (0, 5852)	0.3181294018415408
  (0, 600)	0.29709277499153475
  (0, 2665)	0.13358972114070944
  (0, 6592)	0.29709277499153475
  (0, 5179)	0.2895844778769335
  (0, 5886)	0.33482711194677267
  (0, 3611)	0.2895844778769335
  (0, 3288)	0.29709277499153475
  (1, 7037)	0.270996539088949
  (1, 5982)	0.39072275764596864
  (1, 1873)	0.39072275764596864
  (1, 7917)	0.41123068712662325
  (1, 7969)	0.37617215011720473
  (1, 1979)	0.37617215011720473
  (1, 7806)	0.41123068712662325
  (2, 2734)	0.44300994647648434
  (2, 7157)	0.44300994647648434
  (2, 3705)	0.40524214098364164
  (2, 3655)	0.38314939103579815
  (2, 4430)	0.3453815855429554
  (2, 1916)	0.4209171965286409
  (3, 5520)	0.278681666941605
  :	:
  (2493, 6126)	0.4472135954999579
  (2494, 1252)	0.5232053777216709
  (2494, 6365)	0.4786006931186125
  (2494, 999)	0.4642411919812982
  (2494, 7135)	0.3979841968544478
  (2494, 3704)	0.3511213522294121
  (

In [14]:
import joblib

loaded_content_vector = joblib.load('fe_data/tf-idf-vect-content-1.pkl') 
loaded_title_vector = joblib.load('fe_data/tf-idf-vect-title-1.pkl')

clean_data_tokenized = pd.read_csv("data/merged_data_tokenized2.csv")
clean_data_tokenized = clean_data_tokenized[clean_data_tokenized['is_train'] == 0]
x_content_test = loaded_content_vector.transform(clean_data_tokenized['content_tokenized'])
x_title_test = loaded_title_vector.transform(clean_data_tokenized['title_tokenized'])

In [15]:
from scipy.sparse import hstack


df = pd.read_csv("data/merged_data_tokenized2.csv")
df_train = df[df['is_train'] == 1]

df_test  =  df[df['is_train'] == 0]


X_train = hstack([x_title, x_content])
y_train = df_train['label']


X_test = hstack([x_title_test, x_content_test])
y_test = df_test['label']

# X = hstack([x_title, x_content])
# y = df['label']

In [16]:

import numpy as np
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_classes = label_binarizer.classes_
y_train = np.argmax(y_train, axis=1)

label_binarizer = LabelBinarizer()
y_test = label_binarizer.fit_transform(y_test)
y_test = np.argmax(y_test, axis=1)

# label_binarizer = LabelBinarizer()
# y = label_binarizer.fit_transform(y)
# y_classes = label_binarizer.classes_
# y = np.argmax(y, axis=1)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)
#X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=0)



### Metrics
- **Precision**: Precision is the ratio of true positive predictions to the total number of positive predictions made by the classifier. It measures the accuracy of positive predictions. A high precision indicates that the classifier has a low false positive rate.

- **Recall**: Recall, also known as sensitivity or true positive rate, is the ratio of true positive predictions to the total number of actual positives in the dataset. It measures the ability of the classifier to correctly identify all positive instances. A high recall indicates that the classifier has a low false negative rate.

- **F-score**: The F-score, or F1 score, is the harmonic mean of precision and recall. It provides a single score that balances both precision and recall. The F1 score reaches its best value at 1 and worst at 0.

- **Support**: Support is the number of actual occurrences of each class in the specified dataset. It represents the number of true instances for each class in the dataset.

- **Jaccard Score**: The Jaccard score, also known as the Jaccard similarity coefficient, measures the similarity between two sets by comparing their intersection with their union. In the context of multi-label classification, it calculates the similarity between the predicted labels and the true labels. It ranges from 0 to 1, where 1 indicates perfect overlap between the predicted and true labels.

In [19]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, jaccard_score
from sklearn.metrics import precision_recall_fscore_support as score


    

# Prints Accuracy, Hamming loss and Jaccard score:
def evaluate_model(y_test, y_pred, model_name):
    hamming = []
    precision, recall, fscore, support = score(y_test, y_pred)
    

    # for i, (test, pred) in enumerate(zip(y_test.T, y_pred.T)):
    #     hamming.append(hamming_loss(test, pred))
    
    acc = accuracy_score(y_test, y_pred)
    jacc_sc = jaccard_score(y_test, y_pred, average='weighted')
    
    print("Accuracy: ", acc)
    print("Classifier Used:", str(model_name))
    print(f'Jaccard Score: {jacc_sc:.4f}')
    print("\n")
    
    metric_df = pd.DataFrame(data=[precision, recall, fscore,support],
                         index=["Precision", "Recall", "F-1 score","True Count"],
                         columns=y_classes)
    metric_df.to_csv("models/models_results/"+str(model_name)+"_metrics_split_data.xlsx")
    
    print(metric_df)




In [20]:


classifiers = {
    
    "Linear SVM": LinearSVC(),
    "SGD": SGDClassifier(n_jobs=-1)}


for name, clf in classifiers.items():
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    metric_df = evaluate_model(y_test, y_pred, name)
    filename = 'models/' + name + '.sav'
    joblib.dump(clf, filename)

C:\Users\yvariozidis\PycharmProjects\Topic-classification\.venv\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy:  0.6811797752808989
Classifier Used: Linear SVM
Jaccard Score: 0.5584


            Astrology  Attractions  Automotive     Beauty  Business&Finance  \
Precision    1.000000     0.970588    0.947368   0.735294          0.588235   
Recall       0.888889     0.634615    0.375000   0.925926          0.740741   
F-1 score    0.941176     0.767442    0.537313   0.819672          0.655738   
True Count  36.000000    52.000000   48.000000  27.000000         27.000000   

              Culture  Education  Family&Relationships  Food&Drink  \
Precision    0.215517   0.960000              0.666667    1.000000   
Recall       0.925926   0.888889              0.592593    0.775000   
F-1 score    0.349650   0.923077              0.627451    0.873239   
True Count  27.000000  27.000000             27.000000   40.000000   

            Healthy Living  ...   Politics  Pop Culture  \
Precision         0.490196  ...   1.000000     1.000000   
Recall            0.925926  ...   0.625000     0.3428

### Down break labels to sublabels - Clustering

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import pandas as pd
clustered_data = pd.DataFrame()

df = pd.read_csv("data/merged_data_tokenized2.csv")
tfidf_matrices = []
tfidf_matrices_2d = []

existing_labels = df['label'].unique()
# Clustering within each existing label
for label in existing_labels:
    # Filter articles belonging to the current label
    label_data = df[df['label'] == label].copy() 
    
    label_data['text'] = label_data['title_tokenized'] + ' ' + label_data['content_tokenized']
    text_data = label_data['text'].tolist()
    
    vectorizer = TfidfVectorizer()  # Adjust parameters as needed
    tfidf_matrix = vectorizer.fit_transform(text_data)
    
    # Clustering (K-means)
    num_clusters = 3  # Adjust number of clusters as needed for each label
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(tfidf_matrix)

    # Assign cluster labels to articles
    cluster_labels = kmeans.labels_

    # Add cluster labels and original labels to the dataset
    label_data['subcluster'] = cluster_labels

    # Append the clustered data to the overall dataframe
    clustered_data = pd.concat([clustered_data, label_data])
    

# Display the distribution of articles across subclusters
print(clustered_data.groupby(['label', 'subcluster']).size())

label           subcluster
Astrology       0             29
                1             48
                2              4
Attractions     0             13
                1             12
                              ..
Travel          1             46
                2             19
Viral Articles  0              5
                1             25
                2             68
Length: 63, dtype: int64
